### Chat With Langraph

In [ ]:
!pip install langgraph langsmith

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [langgraph]/5 [langgraph]prebuilt]


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
groq_api_key = os.getenv("GROQ_API_KEY")


In [9]:
langsmith = os.getenv("LANGCHAIN_API_KEY")

In [10]:
os.environ["LANGCHAIN_API_KEY"] = langsmith
os.environ["LANGCHAIN_TRACING_V2"] = "True"
os.environ["LANGCHAIN_PROJECT"] =  "CourseLanggraph"

In [11]:
from langchain_groq import ChatGroq